In [173]:
import pandas as pd
import random
import functools
import itertools

In [6]:
results = {}

def add_result(k, sents, score=0):
    results.update({k: {'sents':sents, 'score':score}})

In [7]:
entities = pd.read_csv('.data/entities.csv')
docs = entities[entities['type'] == 'doctor']['label'].values
services = entities[entities['type'] == 'service']['label'].values

In [157]:
from bs4 import BeautifulSoup
import requests
import re

mode_temps = ['Indicative_Present', 'Indicative_Imperfect', 'Conditional_Present', 'Conditional_First past']
pronoms = [0, 2, 3, 5]

def is_bloc(tag):
    return tag.has_attr('class') and ('modeBloc' in tag.attrs['class'] or 'conjugBloc' in tag.attrs['class'])

def get_verb_conjugate(verb):
    sents = []
    res = requests.get(f'https://leconjugueur.lefigaro.fr/french/verb/{verb}.html')
    print(f'{verb} : {res.status_code}')
    soup = BeautifulSoup(res.text)
    mode = ''
    for tag in soup.find_all(is_bloc):
        if 'modeBloc' in tag.attrs['class']:
            mode = tag.text
        else:
            temps = tag.find('div', attrs={'class':'tempsBloc'})
            if temps and f'{mode}_{temps.text}' in mode_temps:
                text = list(tag.children)[1]
                text = re.sub(r'<\/?(b|p)>', '', str(text))
                sents += [sent for i, sent in enumerate(text.split('<br/>')) if i in pronoms]
    return sents
            

In [182]:
infinitive_verbs = ['demander', 'vouloir', 'reclamer', 'aimer', 'desirer', 'aspirer', 'esperer', 'exiger', 'souhaiter', 'rever']

verbs = {}
for verb in infinitive_verbs:
    verbs[verb] = get_verb_conjugate(verb)

demander : 200
vouloir : 200
reclamer : 200
aimer : 200
desirer : 200
aspirer : 200
esperer : 200
exiger : 200
souhaiter : 200
rever : 200


contacts = ['m\'adresser',
            'discuter avec le', 'converser avec le', 'm\'entretenir avec', 'dialoguer avec',
            'joindre', 'avoir', 'que vous me passiez']


In [194]:
wants = []
for verb in ['vouloir', 'aimer', 'desirer', 'aspirer', 'souhaiter', 'esperer']:
    wants += verbs[verb]

for verb in ['reclamer', 'exiger', 'rever']:
    wants += [f'{v} de' for v in verbs[verb]]
    
for verb in ['demander']:
    wants += [f'{v} à' for v in verbs[verb]]
    

contacts = ['parler', 'causer', 'telephoner'] 
wichs = ['avec le docteur', 'avec la docteur', 'au docteur', 
 'avec le médecin', 'avec la médecin', 'au médecin',
 'avec Mr', 'avec Mme', 'à Monsieur', 'à Madame',
 'à Mr', 'à Mme', 'à Monsieur', 'à Madame']
targets = [doc.split()[0] for doc in docs] + [doc for doc in docs]


In [195]:
sents = []
for want in wants:
    for contact in contacts:
        for wich in wichs:
            for target in targets:
                sents.append(f'{want} {contact} {wich} {target}')
len(sents)

403200

In [196]:
rsents = list(sents)
random.shuffle(rsents)
rsents[:25]

['je rêve de parler à Mme AL',
 'ils souhaiteraient causer au médecin BLIN',
 'nous réclamons de causer à Mr BADO',
 'il aurait désiré causer à Mr BAUER',
 'ils exigeaient de telephoner à Mr CADOT Bernard',
 "j'aimerais telephoner à Monsieur BENHAMOU",
 'ils réclameraient de telephoner avec Mme BRYON Dominique',
 'il aurait réclamé de causer avec Mr ABOAB',
 'je veux causer à Mr BORIE',
 'nous aurions rêvé de telephoner au docteur ALLARD',
 'nous désirerions parler avec la médecin BRUN Laurent',
 "j'aurais demandé à telephoner avec la médecin BLAVIER Bénédicte",
 'il souhaite causer au médecin ABOAB Jennifer',
 'nous aimons parler avec la médecin BRUN',
 "j'aimerais causer à Monsieur BADO",
 'je voulais telephoner à Mr BENHAMOU Claude',
 'nous aspirerions parler à Madame ALLARD Anne',
 'il souhaiterait causer avec le médecin ABOUCAYA Jean-Paul',
 'nous aurions demandé à telephoner avec le médecin ABOAB Jennifer',
 'nous réclamions de parler à Mr BLIN Jean-Louis',
 'je voudrais telephon